# Multi-tool Agent

In our previous SQL Agents notebooks, we saw that our agents had some difficulty generating the correct SQL queries to answer some questions. We will now look at creating an agent that uses tools to get data from the database using predefined SQL queries, and see if this yields better results. We will use the same North Carolina General Statutes dataset.

In [1]:
# VS Code's Jupyter extension doesn't support loading .envrc, so if you're using VS Code, we load it here.

import sys

if ".." not in sys.path:
    sys.path.insert(0, "..")

from utils import load_envrc

load_envrc("../../.envrc")

In [2]:
import pandas as pd

df = pd.read_csv("input/nc_general_statutes.csv.gz")
df.head(5)

,chapter_number,chapter_title,article_number,article,section_number,text,source_url
0,1,Civil Procedure,1,Definitions,1-1,§ 1-1. Remedies.\nRemedies in the courts of j...,https://www.ncleg.gov/EnactedLegislation/Statu...
1,1,Civil Procedure,1,Definitions,1-2,§ 1-2. Actions.\nAn action is an ordinary pro...,https://www.ncleg.gov/EnactedLegislation/Statu...
2,1,Civil Procedure,1,Definitions,1-3,§ 1-3. Special proceedings.\nEvery other reme...,https://www.ncleg.gov/EnactedLegislation/Statu...
3,1,Civil Procedure,1,Definitions,1-4,§ 1-4. Kinds of actions.\nActions are of two ...,https://www.ncleg.gov/EnactedLegislation/Statu...
4,1,Civil Procedure,1,Definitions,1-5,§ 1-5. Criminal action.\nA criminal action is...,https://www.ncleg.gov/EnactedLegislation/Statu...


In [3]:
# Now let's load the data into a PostgreSQL database.

import os
from sqlalchemy import create_engine

pg_engine = create_engine(os.getenv("DATABASE_URL"))

df.to_sql("nc_general_statutes", pg_engine, if_exists="replace", index=False)

-1

### Query the data

Let's think about different ways we can query the data. Some simple examples include:

- "How many chapters are there in total?"
- "What is the title of Chapter 15A?"
- "List all statutes in Chapter 15A."
- "What is the text of statute 15A-145?"

We can obviously ask more complex questions, but these simple ones will do for now. Let's try to answer these questions using SQL queries.

In [4]:
# "How many chapters are there in total?"

pd.read_sql("SELECT COUNT(DISTINCT chapter_number) FROM nc_general_statutes", pg_engine)

,count
0,395


In [5]:
# "What is the title of Chapter 15A?"

pd.read_sql(
    "SELECT DISTINCT chapter_title FROM nc_general_statutes WHERE chapter_number = '15A'", pg_engine
)

,chapter_title
0,Criminal Procedure Act


In [6]:
# "List all articles in Chapter 15A."

pd.read_sql(
    """
    SELECT DISTINCT 
        CAST(REGEXP_REPLACE(article_number, '[^0-9]', '', 'g') AS INTEGER) AS article_number_sort
        , article_number
        , article FROM nc_general_statutes
    WHERE chapter_number = '15A'
    """,
    pg_engine,
    dtype={"article_number_sort": "Int64"},
)

,article_number_sort,article_number,article
0,1,1,Definitions and General Provisions
1,2,2,Jurisdiction
2,3,3,Venue
3,4,4,Entry and Withdrawal of Attorney in Criminal Case
4,5,5,Expunction of Records
...,...,...,...
99,91,91,Appeal to Appellate Division
100,92,92,North Carolina Innocence Inquiry Commission
101,100,100,Capital Punishment
102,101,101,North Carolina Racial Justice Act


We will use the SQL queries above as the basis for some of our tools.

## Creating the Agent

Now let's create a multi-tool agent that can answer questions about the North Carolina General Statutes. In addition to the questions above, ideally the agent can take certain questions further, for example: "Explain like I'm 5: Statute 15A-145".

In [7]:
import pandas as pd
from pydantic import BaseModel
from pydantic_ai import Agent
from rich import print as rich_print

In [28]:
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.ollama import OllamaProvider

model = OpenAIChatModel(
    model_name="llama3.2", provider=OllamaProvider(base_url="http://localhost:11434/v1")
)
# model = "openai:gpt-4o"
model = "openai:gpt-4.1-mini"
# model = "openai:gpt-4.1-nano"

Let's create Pydantic models based on our data and the types of questions that we expect our agent to be able to answer. Some of our tools will either take these as inputs or return them as outputs.

In [9]:
class Chapter(BaseModel):
    """A chapter in the North Carolina General Statutes."""

    chapter_number: str
    chapter_title: str


class Article(BaseModel):
    """An article in the North Carolina General Statutes."""

    article_number: str | None
    article: str | None


class Section(BaseModel):
    """A section or statute in the North Carolina General Statutes."""

    section_number: str
    text: str
    source_url: str

### The tools

We'll define a number of tools, based on the questions we want our agent to be able to answer. These will be simple Python functions that don't take any [dependencies](https://ai.pydantic.dev/dependencies) for now.

In [10]:
def list_chapters(chapter_number: str | None = None) -> list[Chapter]:
    """Get all chapters or a specific chapter by chapter number.

    Args:
        chapter_number: The chapter number to get (optional)."""
    sql = "SELECT DISTINCT chapter_number, chapter_title FROM nc_general_statutes"
    if chapter_number:
        sql += f" WHERE chapter_number = '{chapter_number}'"
    df = pd.read_sql(sql, pg_engine)
    return [Chapter(**row.to_dict()) for index, row in df.iterrows()]

In [11]:
chapters = list_chapters()
rich_print(chapters[:3])
print(f"Total number of chapters: {len(chapters)}")

[
    Chapter(chapter_number='114', chapter_title='Department of Justice'),
    Chapter(chapter_number='74E', chapter_title='Company Police Act'),
    Chapter(chapter_number='22A', chapter_title='Signatures')
]

Total number of chapters: 395


In [12]:
rich_print(list_chapters("15A"))

[Chapter(chapter_number='15A', chapter_title='Criminal Procedure Act')]

In [13]:
def get_chapters_count() -> int:
    """Get the total number of chapters."""
    df = pd.read_sql("SELECT COUNT(DISTINCT chapter_number) FROM nc_general_statutes", pg_engine)
    return int(df.loc[0, "count"])

In [14]:
get_chapters_count()

395

In [15]:
def list_articles(chapter_number: str | None = None) -> list[Article]:
    """List all articles.

    Args:
        chapter_number: The chapter number for which to list articles (optional).
    """
    sql = """
    SELECT DISTINCT 
        CAST(REGEXP_REPLACE(article_number, '[^0-9]', '', 'g') AS INTEGER) AS article_number_sort
        , article_number
        , article
    FROM nc_general_statutes
    """
    params = None
    if chapter_number:
        sql += " WHERE chapter_number = %s"
        params = [(chapter_number,)]
    sql += " ORDER BY article_number_sort, article_number"
    df = pd.read_sql(sql, pg_engine, params=params).drop("article_number_sort", axis=1)
    return [Article(**row.to_dict()) for index, row in df.iterrows()]

In [16]:
articles = list_articles()
rich_print(articles[:3])
print(f"Total number of articles: {len(articles)}")

[
    Article(article_number='1', article='Child Labor Regulations'),
    Article(article_number='1', article='Determination of Incompetence'),
    Article(article_number='1', article='Bipartisan State Board of Elections and Ethics Enforcement')
]

Total number of articles: 2932


In [17]:
articles = list_articles("15A")
rich_print(articles[:3])
print(f"Total number of articles in chapter 15A: {len(articles)}")

[
    Article(article_number='1', article='Definitions and General Provisions'),
    Article(article_number='2', article='Jurisdiction'),
    Article(article_number='3', article='Venue')
]

Total number of articles in chapter 15A: 104


In [18]:
def list_sections(section_number: str) -> list[Section]:
    """Get sections (statutes) by section number.

    Args:
        section_number: The section number for which to list sections.
    """
    df = pd.read_sql(
        "SELECT section_number, text, source_url FROM nc_general_statutes WHERE section_number = %s",
        pg_engine,
        params=[(section_number,)],
    )
    return [Section(**row.to_dict()) for index, row in df.iterrows()]

In [19]:
rich_print(list_sections("15A-145"))

[
    Section(
        section_number='15A-145',
        text="§ 15A-145. \xa0Expunction of records for first offenders under the age of 18 at the time of 
conviction of misdemeanor; expunction of certain other misdemeanors.\n(a)\tWhenever any person who has not 
previously been convicted of any felony, or misdemeanor other than a traffic violation, under the laws of the 
United States, the laws of this State or any other state, (i) pleads guilty to or is guilty of a misdemeanor other 
than a traffic violation, and the offense was committed before the person attained the age of 18 years, or (ii) 
pleads guilty to or is guilty of a misdemeanor possession of alcohol pursuant to G.S.\xa018B-302(b)(1), and the 
offense was committed before the person attained the age of 21 years, he may file a petition in the court of the 
county where he was convicted for expunction of the misdemeanor from his criminal record. The petition cannot be 
filed earlier than: (i) two years after the date of the conviction, or (ii) the completion of any period of 
probation, whichever occurs later, and the petition shall contain, but not be limited to, the following:\n(1)\tAn 
affidavit by the petitioner that he has been of good behavior for the two-year period since the date of conviction 
of the misdemeanor in question and has not been convicted of any felony, or misdemeanor other than a traffic 
violation, under the laws of the United States or the laws of this State or any other state.\n(2)\tVerified 
affidavits of two persons who are not related to the petitioner or to each other by blood or marriage, that they 
know the character and reputation of the petitioner in the community in which he lives and that his character and 
reputation are good.\n(3)\tA statement that the petition is a motion in the cause in the case wherein the 
petitioner was convicted.\n(4)\tRepealed by Session Laws 2010-174, s. 2, effective October 1, 2010, and applicable 
to petitions for expunctions filed on or after that date.\n(4a)\tAn application on a form approved by the 
Administrative Office of the Courts requesting and authorizing a name-based State and national criminal record 
check by the State Bureau of Investigation using any information required by the Administrative Office of the 
Courts to identify the individual and a search of the confidential record of expunctions maintained by the 
Administrative Office of the Courts. The application shall be filed with the clerk of superior court. The clerk of 
superior court shall forward the application to the State Bureau of Investigation and to the Administrative Office 
of the Courts, which shall conduct the searches and report their findings to the court.\n(5)\tAn affidavit by the 
petitioner that no restitution orders or civil judgments representing amounts ordered for restitution entered 
against him are outstanding.\nThe petition shall be served upon the district attorney of the court wherein the case
was tried resulting in conviction. The district attorney shall have 10 days thereafter in which to file any 
objection thereto and shall be duly notified as to the date of the hearing of the petition.\nThe judge to whom the 
petition is presented is authorized to call upon a probation officer for any additional investigation or 
verification of the petitioner's conduct during the two-year period that he deems desirable.\n(a1)\tNothing in this
section shall be interpreted to allow the expunction of any offense involving impaired driving as defined in 
G.S.\xa020-4.01(24a) or any offense requiring registration pursuant to Article 27A of Chapter 14 of the General 
Statutes, whether or not the person is currently required to register.\n(b)\tIf the court, after hearing, finds 
that the petitioner had remained of good behavior and been free of conviction of any felony or misdemeanor, other 
than a traffic violation, for two years from the date of conviction of the misdemeanor in question, the petitioner 
has no outstanding

### Create the agent

In [20]:
agent = Agent(
    model=model,
    system_prompt=(
        "You are a helpful assitant on North Carolina General Statutes. Use the available tools to answer the user's questions."
        "If the user asks about a 'statute', assume they mean 'section'."
    ),
    tools=[list_chapters, list_articles, get_chapters_count, list_sections],
    retries=2,
)


async def prompt(question):
    result = await agent.run(question)
    print(result.output)

In [21]:
await prompt("What is the title of Chapter 15A?")

The title of Chapter 15A is "Criminal Procedure Act."


Iterating over an agent's graph, using the iterator returned by `Agent.iter()`, will help us to see what the agent is actually doing. If for example the agent does not use a tool that we expected it to use, we can try to make some improvements to the agent's instructions or the docstrings on our tools.

In [22]:
async def prompt_iter(question):
    async with agent.iter(question) as agent_run:
        async for node in agent_run:
            rich_print(node)
        # Print out the final output string at the end
        print("-" * 50)
        print(agent_run.result.output)

In [23]:
await prompt_iter("What is the title of Chapter 15A?")

UserPromptNode(user_prompt='What is the title of Chapter 15A?', instructions_functions=[], system_prompts=("You are
a helpful assitant on North Carolina General Statutes. Use the available tools to answer the user's questions.If 
the user asks about a 'statute', assume they mean 'section'.",), system_prompt_functions=[], 
system_prompt_dynamic_functions={})

ModelRequestNode(request=ModelRequest(parts=[SystemPromptPart(content="You are a helpful assitant on North Carolina
General Statutes. Use the available tools to answer the user's questions.If the user asks about a 'statute', assume
they mean 'section'.", timestamp=datetime.datetime(2025, 11, 5, 16, 56, 11, 910289, tzinfo=datetime.timezone.utc)),
UserPromptPart(content='What is the title of Chapter 15A?', timestamp=datetime.datetime(2025, 11, 5, 16, 56, 11, 
910298, tzinfo=datetime.timezone.utc))]))

CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='list_chapters', 
args='{"chapter_number":"15A"}', tool_call_id='call_XGw1as3y4lVDiCl1mnxU54mv')], 
usage=RequestUsage(input_tokens=204, output_tokens=17, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0,
'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-mini-2025-04-14', 
timestamp=datetime.datetime(2025, 11, 5, 16, 56, 12, tzinfo=TzInfo(UTC)), provider_name='openai', 
provider_details={'finish_reason': 'tool_calls'}, provider_response_id='chatcmpl-CYb5oezWmeSO7O7DkBULLr99LAPg7', 
finish_reason='tool_call'))

ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='list_chapters', 
content=[Chapter(chapter_number='15A', chapter_title='Criminal Procedure Act')], 
tool_call_id='call_XGw1as3y4lVDiCl1mnxU54mv', timestamp=datetime.datetime(2025, 11, 5, 16, 56, 12, 838567, 
tzinfo=datetime.timezone.utc))]))

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='The title of Chapter 15A is "Criminal Procedure
Act."')], usage=RequestUsage(input_tokens=247, output_tokens=15, details={'accepted_prediction_tokens': 0, 
'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-mini-2025-04-14', 
timestamp=datetime.datetime(2025, 11, 5, 16, 56, 13, tzinfo=TzInfo(UTC)), provider_name='openai', 
provider_details={'finish_reason': 'stop'}, provider_response_id='chatcmpl-CYb5p3OEBIp6lMxvjteWjTsJ3RbjO', 
finish_reason='stop'))

End(data=FinalResult(output='The title of Chapter 15A is "Criminal Procedure Act."'))

--------------------------------------------------
The title of Chapter 15A is "Criminal Procedure Act."


Initially, the `list_chapters` tool could not filter the chapters by chapter number. I found that the agent could still answer the above question using the full list of chapters, but it's probably better to filter in the tool and use up less tokens when sending the tool's result back to the model.

In [24]:
await prompt_iter("List all articles in Chapter 15A.")

UserPromptNode(user_prompt='List all articles in Chapter 15A.', instructions_functions=[], system_prompts=("You are
a helpful assitant on North Carolina General Statutes. Use the available tools to answer the user's questions.If 
the user asks about a 'statute', assume they mean 'section'.",), system_prompt_functions=[], 
system_prompt_dynamic_functions={})

ModelRequestNode(request=ModelRequest(parts=[SystemPromptPart(content="You are a helpful assitant on North Carolina
General Statutes. Use the available tools to answer the user's questions.If the user asks about a 'statute', assume
they mean 'section'.", timestamp=datetime.datetime(2025, 11, 5, 16, 56, 14, 389552, tzinfo=datetime.timezone.utc)),
UserPromptPart(content='List all articles in Chapter 15A.', timestamp=datetime.datetime(2025, 11, 5, 16, 56, 14, 
389558, tzinfo=datetime.timezone.utc))]))

CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='list_articles', 
args='{"chapter_number":"15A"}', tool_call_id='call_B1bFYiLvci6H44CWfYUW4ZB7')], 
usage=RequestUsage(input_tokens=203, output_tokens=16, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0,
'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-mini-2025-04-14', 
timestamp=datetime.datetime(2025, 11, 5, 16, 56, 15, tzinfo=TzInfo(UTC)), provider_name='openai', 
provider_details={'finish_reason': 'tool_calls'}, provider_response_id='chatcmpl-CYb5r1qnYKriJch4xPCAP3irZ0D7a', 
finish_reason='tool_call'))

ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='list_articles', 
content=[Article(article_number='1', article='Definitions and General Provisions'), Article(article_number='2', 
article='Jurisdiction'), Article(article_number='3', article='Venue'), Article(article_number='4', article='Entry 
and Withdrawal of Attorney in Criminal Case'), Article(article_number='5', article='Expunction of Records'), 
Article(article_number='6', article='Certificate of Relief'), Article(article_number='7', article='Certificate of 
Relief'), Article(article_number='8', article='Electronic Recording of Interrogations'), 
Article(article_number='8A', article='SBI and State Crime Laboratory Access to View and Analyze Recordings'), 
Article(article_number='9', article='Search and Seizure by Consent'), Article(article_number='10', article='Other 
Searches and Seizures'), Article(article_number='11', article='Search Warrants'), Article(article_number='12', 
article='Pen Registers; Trap and Trace Devices'), Article(article_number='13', article='DNA Database and 
Databank'), Article(article_number='14', article='Nontestimonial Identification'), Article(article_number='14A', 
article='Eyewitness Identification Reform Act'), Article(article_number='15', article='Urgent Necessity'), 
Article(article_number='16', article='Electronic Surveillance'), Article(article_number='16A', 
article='Discontinuation of Telecommunications Services'), Article(article_number='16B', article='Use of Unmanned 
Aircraft Systems'), Article(article_number='17', article='Criminal Process'), Article(article_number='18', 
article='Identification Documents'), Article(article_number='19', article='Identification Documents'), 
Article(article_number='20', article='Arrest'), Article(article_number='21', article='Arrest'), 
Article(article_number='22', article='Arrest'), Article(article_number='23', article='Police Processing and Duties 
upon Arrest'), Article(article_number='24', article='Initial Appearance'), Article(article_number='25', 
article='Commitment'), Article(article_number='26', article='Bail'), Article(article_number='27', article='Bail'), 
Article(article_number='28', article='Bail'), Article(article_number='29', article='First Appearance Before 
District Court Judge'), Article(article_number='30', article='Probable-Cause Hearing'), 
Article(article_number='31', article='The Grand Jury and Its Proceedings'), Article(article_number='32', 
article='Indictment and Related Instruments'), Article(article_number='33', article='Indictment and Related 
Instruments'), Article(article_number='34', article='Indictment and Related Instruments'), 
Article(article_number='35', article='Speedy Trial'), Article(article_number='36', article='Special Criminal 
Process for Attendance of Defendants'), Article(article_number='37', article='Uniform Criminal Extradition Act'), 
Article(article_number='38', article='Interstate Agreement on Detainers'), Article(article_number='39', 
article='Other Special Process for Attendance of Defendants'), Article(article_number='40', article='Other Special 
Process for Attendance of Defendants'), Article(article_number='41', article='Other Special Process for Attendance 
of Defendants'), Article(article_number='42', article='Attendance of Witnesses Generally'), 
Article(article_number='43', article='Uniform Act to Secure Attendance of Witnesses from Without a State in 
Criminal Proceedings'), Article(article_number='44', article='Securing Attendance of Prisoners as Witnesses'), 
Article(article_number='45', article='Fair Treatment for Certain Victims and Witnesses'), 
Article(article_number='46', article="Crime Victims' Rights Act"), Article(article_number='47', article="Crime 
Victims' Rights Act"), Article(article_number='48', article='Discovery in the Superior Court'), 
Article(article_number='49', article='Pleadings and Joinder'), Article(article_number='50', article='Voluntary 
Dismissal'), Article(article_number='51', article='Ar

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content="The articles in Chapter 15A are:\n\n1. 
Definitions and General Provisions\n2. Jurisdiction\n3. Venue\n4. Entry and Withdrawal of Attorney in Criminal 
Case\n5. Expunction of Records\n6. Certificate of Relief\n7. Certificate of Relief\n8. Electronic Recording of 
Interrogations\n8A. SBI and State Crime Laboratory Access to View and Analyze Recordings\n9. Search and Seizure by 
Consent\n10. Other Searches and Seizures\n11. Search Warrants\n12. Pen Registers; Trap and Trace Devices\n13. DNA 
Database and Databank\n14. Nontestimonial Identification\n14A. Eyewitness Identification Reform Act\n15. Urgent 
Necessity\n16. Electronic Surveillance\n16A. Discontinuation of Telecommunications Services\n16B. Use of Unmanned 
Aircraft Systems\n17. Criminal Process\n18. Identification Documents\n19. Identification Documents\n20. Arrest\n21.
Arrest\n22. Arrest\n23. Police Processing and Duties upon Arrest\n24. Initial Appearance\n25. Commitment\n26. 
Bail\n27. Bail\n28. Bail\n29. First Appearance Before District Court Judge\n30. Probable-Cause Hearing\n31. The 
Grand Jury and Its Proceedings\n32. Indictment and Related Instruments\n33. Indictment and Related Instruments\n34.
Indictment and Related Instruments\n35. Speedy Trial\n36. Special Criminal Process for Attendance of 
Defendants\n37. Uniform Criminal Extradition Act\n38. Interstate Agreement on Detainers\n39. Other Special Process 
for Attendance of Defendants\n40. Other Special Process for Attendance of Defendants\n41. Other Special Process for
Attendance of Defendants\n42. Attendance of Witnesses Generally\n43. Uniform Act to Secure Attendance of Witnesses 
from Without a State in Criminal Proceedings\n44. Securing Attendance of Prisoners as Witnesses\n45. Fair Treatment
for Certain Victims and Witnesses\n46. Crime Victims' Rights Act\n47. Crime Victims' Rights Act\n48. Discovery in 
the Superior Court\n49. Pleadings and Joinder\n50. Voluntary Dismissal\n51. Arraignment\n52. Motions Practice\n53. 
Motion to Suppress Evidence\n54. Reliability of In-Custody Informant Statements\n55. Reliability of In-Custody 
Informant Statements\n56. Incapacity to Proceed\n57. Pleas\n58. Procedures Relating to Guilty Pleas in Superior 
Court\n58A. Procedures Relating to Felony Guilty Pleas in District Court\n59. Maintenance of Order in the 
Courtroom\n60. Maintenance of Order in the Courtroom\n61. Granting of Immunity to Witnesses\n62. Mistrial\n63. 
Mistrial\n64. Mistrial\n65. In General\n66. Procedure for Hearing and Disposition of Infractions\n71. Right to 
Trial by Jury\n72. Selecting and Impaneling the Jury\n73. Criminal Jury Trial in Superior Court\n74. Criminal Jury 
Trial in Superior Court\n75. Criminal Jury Trial in Superior Court\n76. Criminal Jury Trial in Superior Court\n77. 
Criminal Jury Trial in Superior Court\n78. Order of Commitment to Imprisonment\n79. Order of Commitment to 
Imprisonment\n80. Defendants Found Not Guilty by Reason of Insanity\n81. General Sentencing Provisions\n81A. 
Sentencing Persons Convicted of Felonies\n81B. Structured Sentencing of Persons Convicted of Crimes\n81C. 
Restitution\n81D. Permanent No Contact Order Against Convicted Sex Offender\n82. Probation\n83. Imprisonment\n84. 
Fines\n84A. Post-Release Supervision\n84B. Medical Release of Inmates\n85. Parole\n85A. Parole of Certain Convicted
Felons\n85B. Review of Sentences of Life Imprisonment Without Parole\n86. Reports of Dispositions of Criminal 
Cases\n87. Reports of Dispositions of Criminal Cases\n88. Post-Trial Motions and Appeal\n89. Motion for Appropriate
Relief and Other Post-Trial Relief\n90. Appeals from Magistrates and District Court Judges\n91. Appeal to Appellate
Division\n92. North Carolina Innocence Inquiry Commission\n100. Capital Punishment\n101. North Carolina Racial 
Justice Act\nArticle IX")], usage=RequestUsage(input_tokens=1675, output_tokens=912, 
details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning

End(
    data=FinalResult(output="The articles in Chapter 15A are:\n\n1. Definitions and General Provisions\n2. 
Jurisdiction\n3. Venue\n4. Entry and Withdrawal of Attorney in Criminal Case\n5. Expunction of Records\n6. 
Certificate of Relief\n7. Certificate of Relief\n8. Electronic Recording of Interrogations\n8A. SBI and State Crime
Laboratory Access to View and Analyze Recordings\n9. Search and Seizure by Consent\n10. Other Searches and 
Seizures\n11. Search Warrants\n12. Pen Registers; Trap and Trace Devices\n13. DNA Database and Databank\n14. 
Nontestimonial Identification\n14A. Eyewitness Identification Reform Act\n15. Urgent Necessity\n16. Electronic 
Surveillance\n16A. Discontinuation of Telecommunications Services\n16B. Use of Unmanned Aircraft Systems\n17. 
Criminal Process\n18. Identification Documents\n19. Identification Documents\n20. Arrest\n21. Arrest\n22. 
Arrest\n23. Police Processing and Duties upon Arrest\n24. Initial Appearance\n25. Commitment\n26. Bail\n27. 
Bail\n28. Bail\n29. First Appearance Before District Court Judge\n30. Probable-Cause Hearing\n31. The Grand Jury 
and Its Proceedings\n32. Indictment and Related Instruments\n33. Indictment and Related Instruments\n34. Indictment
and Related Instruments\n35. Speedy Trial\n36. Special Criminal Process for Attendance of Defendants\n37. Uniform 
Criminal Extradition Act\n38. Interstate Agreement on Detainers\n39. Other Special Process for Attendance of 
Defendants\n40. Other Special Process for Attendance of Defendants\n41. Other Special Process for Attendance of 
Defendants\n42. Attendance of Witnesses Generally\n43. Uniform Act to Secure Attendance of Witnesses from Without a
State in Criminal Proceedings\n44. Securing Attendance of Prisoners as Witnesses\n45. Fair Treatment for Certain 
Victims and Witnesses\n46. Crime Victims' Rights Act\n47. Crime Victims' Rights Act\n48. Discovery in the Superior 
Court\n49. Pleadings and Joinder\n50. Voluntary Dismissal\n51. Arraignment\n52. Motions Practice\n53. Motion to 
Suppress Evidence\n54. Reliability of In-Custody Informant Statements\n55. Reliability of In-Custody Informant 
Statements\n56. Incapacity to Proceed\n57. Pleas\n58. Procedures Relating to Guilty Pleas in Superior Court\n58A. 
Procedures Relating to Felony Guilty Pleas in District Court\n59. Maintenance of Order in the Courtroom\n60. 
Maintenance of Order in the Courtroom\n61. Granting of Immunity to Witnesses\n62. Mistrial\n63. Mistrial\n64. 
Mistrial\n65. In General\n66. Procedure for Hearing and Disposition of Infractions\n71. Right to Trial by Jury\n72.
Selecting and Impaneling the Jury\n73. Criminal Jury Trial in Superior Court\n74. Criminal Jury Trial in Superior 
Court\n75. Criminal Jury Trial in Superior Court\n76. Criminal Jury Trial in Superior Court\n77. Criminal Jury 
Trial in Superior Court\n78. Order of Commitment to Imprisonment\n79. Order of Commitment to Imprisonment\n80. 
Defendants Found Not Guilty by Reason of Insanity\n81. General Sentencing Provisions\n81A. Sentencing Persons 
Convicted of Felonies\n81B. Structured Sentencing of Persons Convicted of Crimes\n81C. Restitution\n81D. Permanent 
No Contact Order Against Convicted Sex Offender\n82. Probation\n83. Imprisonment\n84. Fines\n84A. Post-Release 
Supervision\n84B. Medical Release of Inmates\n85. Parole\n85A. Parole of Certain Convicted Felons\n85B. Review of 
Sentences of Life Imprisonment Without Parole\n86. Reports of Dispositions of Criminal Cases\n87. Reports of 
Dispositions of Criminal Cases\n88. Post-Trial Motions and Appeal\n89. Motion for Appropriate Relief and Other 
Post-Trial Relief\n90. Appeals from Magistrates and District Court Judges\n91. Appeal to Appellate Division\n92. 
North Carolina Innocence Inquiry Commission\n100. Capital Punishment\n101. North Carolina Racial Justice 
Act\nArticle IX")
)

--------------------------------------------------
The articles in Chapter 15A are:

1. Definitions and General Provisions
2. Jurisdiction
3. Venue
4. Entry and Withdrawal of Attorney in Criminal Case
5. Expunction of Records
6. Certificate of Relief
7. Certificate of Relief
8. Electronic Recording of Interrogations
8A. SBI and State Crime Laboratory Access to View and Analyze Recordings
9. Search and Seizure by Consent
10. Other Searches and Seizures
11. Search Warrants
12. Pen Registers; Trap and Trace Devices
13. DNA Database and Databank
14. Nontestimonial Identification
14A. Eyewitness Identification Reform Act
15. Urgent Necessity
16. Electronic Surveillance
16A. Discontinuation of Telecommunications Services
16B. Use of Unmanned Aircraft Systems
17. Criminal Process
18. Identification Documents
19. Identification Documents
20. Arrest
21. Arrest
22. Arrest
23. Police Processing and Duties upon Arrest
24. Initial Appearance
25. Commitment
26. Bail
27. Bail
28. Bail
29. First Appea

In [25]:
await prompt_iter("How many chapters are there in total?")

UserPromptNode(user_prompt='How many chapters are there in total?', instructions_functions=[], system_prompts=("You
are a helpful assitant on North Carolina General Statutes. Use the available tools to answer the user's 
questions.If the user asks about a 'statute', assume they mean 'section'.",), system_prompt_functions=[], 
system_prompt_dynamic_functions={})

ModelRequestNode(request=ModelRequest(parts=[SystemPromptPart(content="You are a helpful assitant on North Carolina
General Statutes. Use the available tools to answer the user's questions.If the user asks about a 'statute', assume
they mean 'section'.", timestamp=datetime.datetime(2025, 11, 5, 16, 56, 30, 442977, tzinfo=datetime.timezone.utc)),
UserPromptPart(content='How many chapters are there in total?', timestamp=datetime.datetime(2025, 11, 5, 16, 56, 
30, 442985, tzinfo=datetime.timezone.utc))]))

CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='get_chapters_count', args='{}', 
tool_call_id='call_Evhzq3KgN3t4FTYHtUih8GMQ')], usage=RequestUsage(input_tokens=202, output_tokens=12, 
details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 
0}), model_name='gpt-4.1-mini-2025-04-14', timestamp=datetime.datetime(2025, 11, 5, 16, 56, 31, 
tzinfo=TzInfo(UTC)), provider_name='openai', provider_details={'finish_reason': 'tool_calls'}, 
provider_response_id='chatcmpl-CYb67OmXTqxWtPHOGvB81hWFTKSTB', finish_reason='tool_call'))

ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='get_chapters_count', content=395, 
tool_call_id='call_Evhzq3KgN3t4FTYHtUih8GMQ', timestamp=datetime.datetime(2025, 11, 5, 16, 56, 32, 198112, 
tzinfo=datetime.timezone.utc))]))

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='There are a total of 395 chapters in the North 
Carolina General Statutes.')], usage=RequestUsage(input_tokens=225, output_tokens=17, 
details={'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 
0}), model_name='gpt-4.1-mini-2025-04-14', timestamp=datetime.datetime(2025, 11, 5, 16, 56, 32, 
tzinfo=TzInfo(UTC)), provider_name='openai', provider_details={'finish_reason': 'stop'}, 
provider_response_id='chatcmpl-CYb68PhKW34sUKs8eIHgZgJOcBPk8', finish_reason='stop'))

End(data=FinalResult(output='There are a total of 395 chapters in the North Carolina General Statutes.'))

--------------------------------------------------
There are a total of 395 chapters in the North Carolina General Statutes.


In [26]:
await prompt_iter("What is the text of statute 15A-145?")

UserPromptNode(user_prompt='What is the text of statute 15A-145?', instructions_functions=[], system_prompts=("You 
are a helpful assitant on North Carolina General Statutes. Use the available tools to answer the user's 
questions.If the user asks about a 'statute', assume they mean 'section'.",), system_prompt_functions=[], 
system_prompt_dynamic_functions={})

ModelRequestNode(request=ModelRequest(parts=[SystemPromptPart(content="You are a helpful assitant on North Carolina
General Statutes. Use the available tools to answer the user's questions.If the user asks about a 'statute', assume
they mean 'section'.", timestamp=datetime.datetime(2025, 11, 5, 16, 56, 33, 57753, tzinfo=datetime.timezone.utc)), 
UserPromptPart(content='What is the text of statute 15A-145?', timestamp=datetime.datetime(2025, 11, 5, 16, 56, 33,
57758, tzinfo=datetime.timezone.utc))]))

CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='list_sections', 
args='{"section_number":"15A-145"}', tool_call_id='call_BUb0FeUbgNHdO6ck5pcVIR56')], 
usage=RequestUsage(input_tokens=206, output_tokens=18, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0,
'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-mini-2025-04-14', 
timestamp=datetime.datetime(2025, 11, 5, 16, 56, 33, tzinfo=TzInfo(UTC)), provider_name='openai', 
provider_details={'finish_reason': 'tool_calls'}, provider_response_id='chatcmpl-CYb69KZnNK3hjzoXk2mV6OWvOSc5d', 
finish_reason='tool_call'))

ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='list_sections', 
content=[Section(section_number='15A-145', text="§ 15A-145. \xa0Expunction of records for first offenders under the
age of 18 at the time of conviction of misdemeanor; expunction of certain other misdemeanors.\n(a)\tWhenever any 
person who has not previously been convicted of any felony, or misdemeanor other than a traffic violation, under 
the laws of the United States, the laws of this State or any other state, (i) pleads guilty to or is guilty of a 
misdemeanor other than a traffic violation, and the offense was committed before the person attained the age of 18 
years, or (ii) pleads guilty to or is guilty of a misdemeanor possession of alcohol pursuant to 
G.S.\xa018B-302(b)(1), and the offense was committed before the person attained the age of 21 years, he may file a 
petition in the court of the county where he was convicted for expunction of the misdemeanor from his criminal 
record. The petition cannot be filed earlier than: (i) two years after the date of the conviction, or (ii) the 
completion of any period of probation, whichever occurs later, and the petition shall contain, but not be limited 
to, the following:\n(1)\tAn affidavit by the petitioner that he has been of good behavior for the two-year period 
since the date of conviction of the misdemeanor in question and has not been convicted of any felony, or 
misdemeanor other than a traffic violation, under the laws of the United States or the laws of this State or any 
other state.\n(2)\tVerified affidavits of two persons who are not related to the petitioner or to each other by 
blood or marriage, that they know the character and reputation of the petitioner in the community in which he lives
and that his character and reputation are good.\n(3)\tA statement that the petition is a motion in the cause in the
case wherein the petitioner was convicted.\n(4)\tRepealed by Session Laws 2010-174, s. 2, effective October 1, 
2010, and applicable to petitions for expunctions filed on or after that date.\n(4a)\tAn application on a form 
approved by the Administrative Office of the Courts requesting and authorizing a name-based State and national 
criminal record check by the State Bureau of Investigation using any information required by the Administrative 
Office of the Courts to identify the individual and a search of the confidential record of expunctions maintained 
by the Administrative Office of the Courts. The application shall be filed with the clerk of superior court. The 
clerk of superior court shall forward the application to the State Bureau of Investigation and to the 
Administrative Office of the Courts, which shall conduct the searches and report their findings to the 
court.\n(5)\tAn affidavit by the petitioner that no restitution orders or civil judgments representing amounts 
ordered for restitution entered against him are outstanding.\nThe petition shall be served upon the district 
attorney of the court wherein the case was tried resulting in conviction. The district attorney shall have 10 days 
thereafter in which to file any objection thereto and shall be duly notified as to the date of the hearing of the 
petition.\nThe judge to whom the petition is presented is authorized to call upon a probation officer for any 
additional investigation or verification of the petitioner's conduct during the two-year period that he deems 
desirable.\n(a1)\tNothing in this section shall be interpreted to allow the expunction of any offense involving 
impaired driving as defined in G.S.\xa020-4.01(24a) or any offense requiring registration pursuant to Article 27A 
of Chapter 14 of the General Statutes, whether or not the person is currently required to register.\n(b)\tIf the 
court, after hearing, finds that the petitioner had remained of good behavior and been free of conviction of any 
felony or misdemeanor, other than a traffic violation, for two years from the date of conv

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content="The text of statute 15A-145 is as follows:\n\n§
15A-145. Expunction of records for first offenders under the age of 18 at the time of conviction of misdemeanor; 
expunction of certain other misdemeanors.\n\n(a) Whenever any person who has not previously been convicted of any 
felony, or misdemeanor other than a traffic violation, under the laws of the United States, the laws of this State 
or any other state, (i) pleads guilty to or is guilty of a misdemeanor other than a traffic violation, and the 
offense was committed before the person attained the age of 18 years, or (ii) pleads guilty to or is guilty of a 
misdemeanor possession of alcohol pursuant to G.S. 18B-302(b)(1), and the offense was committed before the person 
attained the age of 21 years, he may file a petition in the court of the county where he was convicted for 
expunction of the misdemeanor from his criminal record. The petition cannot be filed earlier than: (i) two years 
after the date of the conviction, or (ii) the completion of any period of probation, whichever occurs later, and 
the petition shall contain, but not be limited to, the following:\n(1) An affidavit by the petitioner that he has 
been of good behavior for the two-year period since the date of conviction of the misdemeanor in question and has 
not been convicted of any felony, or misdemeanor other than a traffic violation, under the laws of the United 
States or the laws of this State or any other state.\n(2) Verified affidavits of two persons who are not related to
the petitioner or to each other by blood or marriage, that they know the character and reputation of the petitioner
in the community in which he lives and that his character and reputation are good.\n(3) A statement that the 
petition is a motion in the cause in the case wherein the petitioner was convicted.\n(4) Repealed by Session Laws 
2010-174, s. 2, effective October 1, 2010, and applicable to petitions for expunctions filed on or after that 
date.\n(4a) An application on a form approved by the Administrative Office of the Courts requesting and authorizing
a name-based State and national criminal record check by the State Bureau of Investigation using any information 
required by the Administrative Office of the Courts to identify the individual and a search of the confidential 
record of expunctions maintained by the Administrative Office of the Courts. The application shall be filed with 
the clerk of superior court. The clerk of superior court shall forward the application to the State Bureau of 
Investigation and to the Administrative Office of the Courts, which shall conduct the searches and report their 
findings to the court.\n(5) An affidavit by the petitioner that no restitution orders or civil judgments 
representing amounts ordered for restitution entered against him are outstanding.\nThe petition shall be served 
upon the district attorney of the court wherein the case was tried resulting in conviction. The district attorney 
shall have 10 days thereafter in which to file any objection thereto and shall be duly notified as to the date of 
the hearing of the petition.\nThe judge to whom the petition is presented is authorized to call upon a probation 
officer for any additional investigation or verification of the petitioner's conduct during the two-year period 
that he deems desirable.\n(a1) Nothing in this section shall be interpreted to allow the expunction of any offense 
involving impaired driving as defined in G.S. 20-4.01(24a) or any offense requiring registration pursuant to 
Article 27A of Chapter 14 of the General Statutes, whether or not the person is currently required to 
register.\n(b) If the court, after hearing, finds that the petitioner had remained of good behavior and been free 
of conviction of any felony or misdemeanor, other than a traffic violation, for two years from the date of 
conviction of the misdemeanor in question, t

End(
    data=FinalResult(output="The text of statute 15A-145 is as follows:\n\n§ 15A-145. Expunction of records for 
first offenders under the age of 18 at the time of conviction of misdemeanor; expunction of certain other 
misdemeanors.\n\n(a) Whenever any person who has not previously been convicted of any felony, or misdemeanor other 
than a traffic violation, under the laws of the United States, the laws of this State or any other state, (i) 
pleads guilty to or is guilty of a misdemeanor other than a traffic violation, and the offense was committed before
the person attained the age of 18 years, or (ii) pleads guilty to or is guilty of a misdemeanor possession of 
alcohol pursuant to G.S. 18B-302(b)(1), and the offense was committed before the person attained the age of 21 
years, he may file a petition in the court of the county where he was convicted for expunction of the misdemeanor 
from his criminal record. The petition cannot be filed earlier than: (i) two years after the date of the 
conviction, or (ii) the completion of any period of probation, whichever occurs later, and the petition shall 
contain, but not be limited to, the following:\n(1) An affidavit by the petitioner that he has been of good 
behavior for the two-year period since the date of conviction of the misdemeanor in question and has not been 
convicted of any felony, or misdemeanor other than a traffic violation, under the laws of the United States or the 
laws of this State or any other state.\n(2) Verified affidavits of two persons who are not related to the 
petitioner or to each other by blood or marriage, that they know the character and reputation of the petitioner in 
the community in which he lives and that his character and reputation are good.\n(3) A statement that the petition 
is a motion in the cause in the case wherein the petitioner was convicted.\n(4) Repealed by Session Laws 2010-174, 
s. 2, effective October 1, 2010, and applicable to petitions for expunctions filed on or after that date.\n(4a) An 
application on a form approved by the Administrative Office of the Courts requesting and authorizing a name-based 
State and national criminal record check by the State Bureau of Investigation using any information required by the
Administrative Office of the Courts to identify the individual and a search of the confidential record of 
expunctions maintained by the Administrative Office of the Courts. The application shall be filed with the clerk of
superior court. The clerk of superior court shall forward the application to the State Bureau of Investigation and 
to the Administrative Office of the Courts, which shall conduct the searches and report their findings to the 
court.\n(5) An affidavit by the petitioner that no restitution orders or civil judgments representing amounts 
ordered for restitution entered against him are outstanding.\nThe petition shall be served upon the district 
attorney of the court wherein the case was tried resulting in conviction. The district attorney shall have 10 days 
thereafter in which to file any objection thereto and shall be duly notified as to the date of the hearing of the 
petition.\nThe judge to whom the petition is presented is authorized to call upon a probation officer for any 
additional investigation or verification of the petitioner's conduct during the two-year period that he deems 
desirable.\n(a1) Nothing in this section shall be interpreted to allow the expunction of any offense involving 
impaired driving as defined in G.S. 20-4.01(24a) or any offense requiring registration pursuant to Article 27A of 
Chapter 14 of the General Statutes, whether or not the person is currently required to register.\n(b) If the court,
after hearing, finds that the petitioner had remained of good behavior and been free of conviction of any felony or
misdemeanor, other than a traffic violation, for two years from the date of conviction of the misdemeanor in 
question, the petitioner has no outstanding re

--------------------------------------------------
The text of statute 15A-145 is as follows:

§ 15A-145. Expunction of records for first offenders under the age of 18 at the time of conviction of misdemeanor; expunction of certain other misdemeanors.

(a) Whenever any person who has not previously been convicted of any felony, or misdemeanor other than a traffic violation, under the laws of the United States, the laws of this State or any other state, (i) pleads guilty to or is guilty of a misdemeanor other than a traffic violation, and the offense was committed before the person attained the age of 18 years, or (ii) pleads guilty to or is guilty of a misdemeanor possession of alcohol pursuant to G.S. 18B-302(b)(1), and the offense was committed before the person attained the age of 21 years, he may file a petition in the court of the county where he was convicted for expunction of the misdemeanor from his criminal record. The petition cannot be filed earlier than: (i) two years after

In [27]:
await prompt_iter("Explain like I'm 5: Statute 15A-145")

UserPromptNode(user_prompt="Explain like I'm 5: Statute 15A-145", instructions_functions=[], system_prompts=("You 
are a helpful assitant on North Carolina General Statutes. Use the available tools to answer the user's 
questions.If the user asks about a 'statute', assume they mean 'section'.",), system_prompt_functions=[], 
system_prompt_dynamic_functions={})

ModelRequestNode(request=ModelRequest(parts=[SystemPromptPart(content="You are a helpful assitant on North Carolina
General Statutes. Use the available tools to answer the user's questions.If the user asks about a 'statute', assume
they mean 'section'.", timestamp=datetime.datetime(2025, 11, 5, 16, 57, 7, 844196, tzinfo=datetime.timezone.utc)), 
UserPromptPart(content="Explain like I'm 5: Statute 15A-145", timestamp=datetime.datetime(2025, 11, 5, 16, 57, 7, 
844201, tzinfo=datetime.timezone.utc))]))

CallToolsNode(model_response=ModelResponse(parts=[ToolCallPart(tool_name='list_sections', 
args='{"section_number":"15A-145"}', tool_call_id='call_E7PxWJyqWBS5XgmSJN32tFEn')], 
usage=RequestUsage(input_tokens=207, output_tokens=18, details={'accepted_prediction_tokens': 0, 'audio_tokens': 0,
'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-mini-2025-04-14', 
timestamp=datetime.datetime(2025, 11, 5, 16, 57, 8, tzinfo=TzInfo(UTC)), provider_name='openai', 
provider_details={'finish_reason': 'tool_calls'}, provider_response_id='chatcmpl-CYb6ij9Ku25M6Nk2v1oQKkixuCXZ7', 
finish_reason='tool_call'))

ModelRequestNode(request=ModelRequest(parts=[ToolReturnPart(tool_name='list_sections', 
content=[Section(section_number='15A-145', text="§ 15A-145. \xa0Expunction of records for first offenders under the
age of 18 at the time of conviction of misdemeanor; expunction of certain other misdemeanors.\n(a)\tWhenever any 
person who has not previously been convicted of any felony, or misdemeanor other than a traffic violation, under 
the laws of the United States, the laws of this State or any other state, (i) pleads guilty to or is guilty of a 
misdemeanor other than a traffic violation, and the offense was committed before the person attained the age of 18 
years, or (ii) pleads guilty to or is guilty of a misdemeanor possession of alcohol pursuant to 
G.S.\xa018B-302(b)(1), and the offense was committed before the person attained the age of 21 years, he may file a 
petition in the court of the county where he was convicted for expunction of the misdemeanor from his criminal 
record. The petition cannot be filed earlier than: (i) two years after the date of the conviction, or (ii) the 
completion of any period of probation, whichever occurs later, and the petition shall contain, but not be limited 
to, the following:\n(1)\tAn affidavit by the petitioner that he has been of good behavior for the two-year period 
since the date of conviction of the misdemeanor in question and has not been convicted of any felony, or 
misdemeanor other than a traffic violation, under the laws of the United States or the laws of this State or any 
other state.\n(2)\tVerified affidavits of two persons who are not related to the petitioner or to each other by 
blood or marriage, that they know the character and reputation of the petitioner in the community in which he lives
and that his character and reputation are good.\n(3)\tA statement that the petition is a motion in the cause in the
case wherein the petitioner was convicted.\n(4)\tRepealed by Session Laws 2010-174, s. 2, effective October 1, 
2010, and applicable to petitions for expunctions filed on or after that date.\n(4a)\tAn application on a form 
approved by the Administrative Office of the Courts requesting and authorizing a name-based State and national 
criminal record check by the State Bureau of Investigation using any information required by the Administrative 
Office of the Courts to identify the individual and a search of the confidential record of expunctions maintained 
by the Administrative Office of the Courts. The application shall be filed with the clerk of superior court. The 
clerk of superior court shall forward the application to the State Bureau of Investigation and to the 
Administrative Office of the Courts, which shall conduct the searches and report their findings to the 
court.\n(5)\tAn affidavit by the petitioner that no restitution orders or civil judgments representing amounts 
ordered for restitution entered against him are outstanding.\nThe petition shall be served upon the district 
attorney of the court wherein the case was tried resulting in conviction. The district attorney shall have 10 days 
thereafter in which to file any objection thereto and shall be duly notified as to the date of the hearing of the 
petition.\nThe judge to whom the petition is presented is authorized to call upon a probation officer for any 
additional investigation or verification of the petitioner's conduct during the two-year period that he deems 
desirable.\n(a1)\tNothing in this section shall be interpreted to allow the expunction of any offense involving 
impaired driving as defined in G.S.\xa020-4.01(24a) or any offense requiring registration pursuant to Article 27A 
of Chapter 14 of the General Statutes, whether or not the person is currently required to register.\n(b)\tIf the 
court, after hearing, finds that the petitioner had remained of good behavior and been free of conviction of any 
felony or misdemeanor, other than a traffic violation, for two years from the date of conv

CallToolsNode(model_response=ModelResponse(parts=[TextPart(content='Statute 15A-145 is about giving a "fresh start"
to young people who made a mistake.\n\nIf someone under 18 years old breaks a rule (a misdemeanor, but not traffic 
rules), or if someone under 21 has alcohol when they shouldn\'t, they can ask a court to erase that mistake from 
their record after a certain time.\n\nThe rule says:\n- This person must not have other serious mistakes before.\n-
They have to wait at least two years and behave well during that time.\n- They need to prove to the court that they
have been good and trustworthy.\n- The court checks and if everything is good, it erases that mistake from their 
record.\n- Once erased, they don’t have to worry about that mistake when asked in the future.\n\nIt\'s like erasing
a mark on a school paper when you’ve shown you\'ve learned from it and are doing better now.')], 
usage=RequestUsage(input_tokens=2071, output_tokens=181, details={'accepted_prediction_tokens': 0, 'audio_tokens': 
0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}), model_name='gpt-4.1-mini-2025-04-14', 
timestamp=datetime.datetime(2025, 11, 5, 16, 57, 9, tzinfo=TzInfo(UTC)), provider_name='openai', 
provider_details={'finish_reason': 'stop'}, provider_response_id='chatcmpl-CYb6jDC80KLQgjHOOgTR3Wopsqs4b', 
finish_reason='stop'))

End(
    data=FinalResult(output='Statute 15A-145 is about giving a "fresh start" to young people who made a 
mistake.\n\nIf someone under 18 years old breaks a rule (a misdemeanor, but not traffic rules), or if someone under
21 has alcohol when they shouldn\'t, they can ask a court to erase that mistake from their record after a certain 
time.\n\nThe rule says:\n- This person must not have other serious mistakes before.\n- They have to wait at least 
two years and behave well during that time.\n- They need to prove to the court that they have been good and 
trustworthy.\n- The court checks and if everything is good, it erases that mistake from their record.\n- Once 
erased, they don’t have to worry about that mistake when asked in the future.\n\nIt\'s like erasing a mark on a 
school paper when you’ve shown you\'ve learned from it and are doing better now.')
)

--------------------------------------------------
Statute 15A-145 is about giving a "fresh start" to young people who made a mistake.

If someone under 18 years old breaks a rule (a misdemeanor, but not traffic rules), or if someone under 21 has alcohol when they shouldn't, they can ask a court to erase that mistake from their record after a certain time.

The rule says:
- This person must not have other serious mistakes before.
- They have to wait at least two years and behave well during that time.
- They need to prove to the court that they have been good and trustworthy.
- The court checks and if everything is good, it erases that mistake from their record.
- Once erased, they don’t have to worry about that mistake when asked in the future.

It's like erasing a mark on a school paper when you’ve shown you've learned from it and are doing better now.
